# Impoting the `libs`

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# The Neural Network

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # Block 1: Initial Convolution Block
        # Input: 28x28 (assuming input images are 28x28), Output: 28x28
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1)       
        self.bn1 = nn.BatchNorm2d(8)

        # Block 2: Second Convolution Block
        # Input: 28x28, Output: 28x28        
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)     
        self.bn2 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)                  
        self.drop = nn.Dropout(0.25)

        # Transition: 1x1 Convolution for Channel Reduction
        # Acts as a Global Average Pooling (GAP) mimic, reducing channels.
        self.conv1x1_a = nn.Conv2d(16, 8, 1)            
        
        # Block 3: Third Convolution Block
        # Input: 14x14, Output: 14x14
        self.conv3 = nn.Conv2d(8, 12, 3, padding=1)      
        self.bn3 = nn.BatchNorm2d(12)
        self.conv4 = nn.Conv2d(12, 16, 3, padding=1)    
        self.bn4 = nn.BatchNorm2d(16)
        self.pool2 = nn.MaxPool2d(2, 2)                 
        self.drop = nn.Dropout(0.25)

        # Transition: 1x1 Convolution for Channel Reduction
        # Further reduces channels for efficiency.
        self.conv1x1_b = nn.Conv2d(16, 8, 1)             

        # Block 4: Final Convolution Block
        # Sequential convolutions reduce spatial dimensions to 1x1.
        # Input: 7x7, Output: 5x5
        self.conv5 = nn.Conv2d(8, 12, 3)                
        self.bn5 = nn.BatchNorm2d(12)
        self.conv6 = nn.Conv2d(12, 16, 3)                
        self.bn6 = nn.BatchNorm2d(16)
        self.conv7 = nn.Conv2d(16, 10, 3)               

    def forward(self, x):
        # Apply the layers defined in __init__ with ReLU activations and pooling where specified.
        x = self.pool1(self.bn2(F.relu(self.conv2(self.bn1(F.relu(self.conv1(x)))))))
        x = self.drop(x)
        x = self.conv1x1_a(x)
        x = self.pool2(self.bn4(F.relu(self.conv4(self.bn3(F.relu(self.conv3(x)))))))
        x = self.drop(x)
        x = self.conv1x1_b(x)
        x = self.bn6(F.relu(self.conv6(self.bn5(F.relu(self.conv5(x))))))
        x = self.conv7(x)
        # Flatten the output for classification
        x = x.view(-1, 10) 
        
        return F.log_softmax(x, dim=1)

# Network `Parameters`

In [3]:
# !pip install torchsummary

from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3           [-1, 16, 28, 28]           1,168
       BatchNorm2d-4           [-1, 16, 28, 28]              32
         MaxPool2d-5           [-1, 16, 14, 14]               0
           Dropout-6           [-1, 16, 14, 14]               0
            Conv2d-7            [-1, 8, 14, 14]             136
            Conv2d-8           [-1, 12, 14, 14]             876
       BatchNorm2d-9           [-1, 12, 14, 14]              24
           Conv2d-10           [-1, 16, 14, 14]           1,744
      BatchNorm2d-11           [-1, 16, 14, 14]              32
        MaxPool2d-12             [-1, 16, 7, 7]               0
          Dropout-13             [-1, 16, 7, 7]               0
           Conv2d-14              [-1, 

# Loading the data using `DataLoader`

In [4]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=True, download=True,
                                                                     transform=transforms.Compose([
                                                                     transforms.RandomRotation(degrees=(-5, 5)),
                                                                     transforms.ToTensor(),
                                                                     transforms.Normalize((0.1307,), (0.3081,))
                                                                     ])),
                                           batch_size=batch_size, 
                                           shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=False, transform=transforms.Compose([
                                                                       transforms.ToTensor(),
                                                                       transforms.Normalize((0.1307, ), (0.3081, ))
                                                                   ])),
                   
                                          batch_size=batch_size, 
                                          shuffle=True, **kwargs)


# Wrapper functions for `train` and `test`

In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# `Running` and `Testing` the model

In [7]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    print(f"Epoch #{epoch}") 
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Epoch #1


loss=0.09622154384851456 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 32.61it/s] 



Test set: Average loss: 0.0487, Accuracy: 9852/10000 (98.52%)

Epoch #2


loss=0.16819874942302704 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 32.62it/s] 



Test set: Average loss: 0.0369, Accuracy: 9875/10000 (98.75%)

Epoch #3


...


Test set: Average loss: 0.0200, Accuracy: 9934/10000 (99.34%)

Epoch #19


loss=0.0031311791390180588 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 32.68it/s]



Test set: Average loss: 0.0185, Accuracy: 9946/10000 (99.46%)

Epoch #20


loss=0.007915577851235867 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 32.60it/s] 



Test set: Average loss: 0.0178, Accuracy: 9940/10000 (99.40%)

